In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torchaudio
import pandas as pd
import numpy as np
from numpy import ma
import librosa as lib
import librosa.display as display
import matplotlib.pyplot as plt
import os
import time
torch.backends.cudnn.enabled = True

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
import math
def get_sample(feats, left, right, sampleNum):
    feats = feats.permute(1 , 0)
    x = torch.zeros([240])
    x[0:80] = feats[sampleNum - left]
    x[80:160] = feats[sampleNum - left + 1]
    x[160:240] = feats[sampleNum - left + 2]

    return x
ground_truth = ['blank', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
                'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', "''"]#, '.', ",", '?']

def format_label(label, ground_truth, target_len):
    label = label.lower() #convert capital letters to lowercase
    output = torch.zeros([target_len]) #all targets must be the same length for batch training
    for i in range(0, target_len): #leave off the newline character
        if i < len(label):
            if label[i] not in ground_truth: #replace any unaccounted for characters with ' '
                output[i] = ground_truth.index(' ')
            else:
                output[i] = ground_truth.index(label[i])
        else:
            output[i] = -1
    return output.int()

def second_largest(nums): #used for debugging
    largest = -99999999
    secondLargest = -99999999
    index = -1
    Lindex = -1
    for i in range(0,len(nums)):
        if nums[i] > largest:
            secondLargest = largest
            largest = nums[i]
            index = Lindex
            Lindex = i
            
        if nums[i] < largest and nums[i] > secondLargest:
            secondLargest = nums[i]
            index = i
    return secondLargest, index

def transcribe_output(output, trained):
    sentence = []
    seconds = []
    lastMax = -1
    for i in range(0,len(output)):
        maxNum = output[i][0].max(0)[1]
        letter = ground_truth[maxNum]
        if letter == 'blank':
            seconds.append(ground_truth[second_largest(output[i][0])[1]])
            continue
        elif lastMax == maxNum and not trained: #ignore duplicate letters if the network is not fully trained
            continue
        else:
            lastMax = maxNum
            sentence.append(letter)
    return sentence, seconds

In [4]:
dirRoot = "./datasets/LibriSpeech/train-clean-360/"

In [8]:
class LibriSpeech_Dataset(Dataset):
    
    def __init__(self, dirRoot, n_mels, max_len, left_samples, right_samples, skip, max_label_len):
        self.dir_root = dirRoot
        self.n_mels = n_mels
        self.max_len = max_len
        self.left = left_samples
        self.right = right_samples
        self.window = self.left + self.right
        self.skip = skip
        self.max_target_len = max_label_len
        self.outliers = []
        self.running_label_max = 0
        

        
        #first create folder info lists
        self.speaker_list = os.listdir(dirRoot)
        self.chapter_list = []
        for i in self.speaker_list:
            self.chapter_list.append(os.listdir(dirRoot + i + '/'))
        
        #add all file names to a single list
        self.file_names = []
        self.labels = []
        x = 0
        for i in range(0, len(self.speaker_list)): #for each speaker
            for j in range(0, len(self.chapter_list[i])): #for the chapters they read
                file_path = self.speaker_list[i] + '/' + self.chapter_list[i][j] + '/' #set file path prefix
                label_file_name = self.speaker_list[i] + '-' + self.chapter_list[i][j] + '.trans.txt'
                try: 
                    label_file = pd.read_csv(self.dir_root + file_path + label_file_name)
                except FileNotFoundError:
                    print(label_file_name + ' not found')
                    continue
                for z in os.listdir(dirRoot + file_path): #for every audio file in the chapter folder
                    if z[-4:] == 'flac': #skip txt files
                        label_index = z[-9:-5]
                        label_index = int(label_index)
                        if (label_index == 0): #label_file ignores the first label entry
                            continue
                        full_label = label_file.iloc[label_index - 1][0] #0 corresponds to label 1
                        label = full_label.split(' ', 1)
                        if len(label[1]) <= self.max_target_len: #and len(label[1]) > (self.max_target_len- 150):
                            self.labels.append(label[1]) #remove the file info
                            self.file_names.append(file_path + z)
                            if len(label[1]) > self.running_label_max:
                                self.running_label_max = len(label[1])
                                self.max_len_idx = x
                            x += 1
                            
    def order(self):
        ordered_labels = []
        ordered_file_names = []
        minIndex = -1
        for j in range(0, len(self.labels)):
            runningMin = 9999999
            for i in range(0, len(self.labels)):
                if len(self.labels[i]) < runningMin:
                    runningMin = len(self.labels[i])
                    minIndex = i
            ordered_labels.append(self.labels.pop(minIndex))
            ordered_file_names.append(self.file_names.pop(minIndex))
        self.labels = ordered_labels
        self.file_names = ordered_file_names

            
            
    def __getitem__(self, index):
        #data, sr = lib.core.load(self.dir_root + self.file_names[index], sr = None)
        #mel_feats = lib.feature.melspectrogram(y = data, n_mels = self.n_mels, sr = sr,
        #                                       n_fft = 400, hop_length = 160)
        data = torchaudio.load(self.dir_root + self.file_names[index], out = None, normalization = True)
        raw_data = torchaudio.transforms.LC2CL()(data[0])
        mel_feats2 = torchaudio.transforms.MEL2(sr = 16000, n_mels = self.n_mels, ws = 400, hop = 160)(raw_data)
        mel_feats2 = mel_feats2.permute(0, 2, 1)
        mel_feats2 = mel_feats2[0]
        mel_feats2 = lib.core.db_to_power(mel_feats2.numpy(), ref = 1)
        mel_feats = torch.from_numpy(mel_feats2)
        mel_feats = mel_feats.clamp(min = 1e-8)
        #mel_feats = np.clip(mel_feats, 1e-8, None)
        log_feats = torch.log(mel_feats)
        #log_feats = ma.log(mel_feats)
        #log_feats = torch.from_numpy(log_feats) #full log mel filterbank array
        if log_feats.size(1) < self.max_len:
            length = log_feats.size(1)//3 - 1
            padding = torch.zeros([self.n_mels, self.max_len-log_feats.size(1)]).float()
            log_feats = torch.cat([log_feats, padding], dim = 1)
        else:
            length = self.max_len//3 - 1
            self.outliers.append(index)
            log_feats = log_feats.narrow(1, 0, self.max_len)
        #assert log_feats[0].numel() == self.max_len
        formatted_feats = torch.zeros([(log_feats.size(1) - self.left)//self.skip, 240])
        for j in range(self.left, log_feats.size(1)//self.skip): #copy each input window into the final output array
            formatted_feats[j - self.left] = get_sample(log_feats, self.left, self.right, j*self.skip)
        target = format_label(self.labels[index], ground_truth, self.max_target_len)
        return formatted_feats, target, self.labels[index], length
            
    def __len__(self):
        return(len(self.labels))
    
tester = LibriSpeech_Dataset(dirRoot, 80, 1220, 3, 0, 3, 200)


print(len(tester))


temp, target, label, length = tester[tester.max_len_idx]
print(length)
#print(temp.shape)
#print(tester.outliers)

54936
38


In [11]:
timer1 = time.time()
#temp, target, label, length = tester[tester.max_len_idx]

tester.order()
timer2 = time.time()
print(timer2-timer1)

0.024469852447509766


In [13]:
class Net(nn.Module):  
    def __init__(self):
        super(Net, self).__init__()
        self.drop = nn.Dropout(0.4)
        self.drop2 = nn.Dropout(0.4)
        self.lstm = nn.LSTM(input_size = 240, hidden_size = 512, num_layers = 5, bidirectional = True)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 29)
        #self.fc3 = nn.Linear(256, 28)
        
    def forward(self, x): #expected dims: batch, sample, channel, rows, columns
        x = x.permute(1, 0, 2).contiguous()
        x, self.hidden = self.lstm(x, self.hidden)
        x = x.clamp(min = -50, max = 50)
        x = self.drop(x)
        y = self.fc1(x.view(x.size(0) * x.size(1), x.size(2)).contiguous())
        y = self.drop2(y)
        y = self.fc2(y)
        #y = self.fc3(y)
        #self.experimental = y.view(x.size(0), x.size(1), y.size(1)).contiguous()
        return F.log_softmax(y.view(x.size(0), x.size(1), y.size(1)).contiguous(), dim = 2)

    def init_hidden(self, batch_size):
        mean = torch.zeros([10,batch_size,512])
        std = torch.zeros([10,batch_size,512])
        std += (1/512)**(0.5)
        return (torch.normal(mean,std).to(device),
                torch.normal(mean,std).to(device))


model = Net().to(device)
model.hidden = model.init_hidden(4)
test = torch.rand([4,400,240]).to(device)
x = model(test)
print(x.shape)
print(model)

torch.Size([400, 4, 29])
Net(
  (drop): Dropout(p=0.4)
  (drop2): Dropout(p=0.4)
  (lstm): LSTM(240, 512, num_layers=5, bidirectional=True)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=29, bias=True)
)


In [14]:
batch_size = 8
kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu
train_loader = torch.utils.data.DataLoader(tester, batch_size = batch_size, shuffle = False, **kwargs)

In [15]:
optimizer = torch.optim.ASGD(model.parameters(), lr = 0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)
ctc_loss = torch.nn.CTCLoss(blank = 0)

In [16]:
import time    
def train(model, epoch):
    model.train()
    dataLoadStart = time.time()
    for batch_idx, (data, target, labels, lengths) in enumerate(train_loader):
        start = time.time()
        lengths = lengths.to(device)
        #print("Data load time: " + str(start-dataLoadStart))
        #print(data.shape)
        loop_bound = data.size(0)
        optimizer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        model.hidden = model.init_hidden(loop_bound)
        target_lens = torch.zeros([loop_bound]).to(device)
        #data_lens = torch.zeros([loop_bound]).to(device)
        
        out = model(data)
        for i in range(0, loop_bound):
            target_lens[i] = len(labels[i])
            
        loss = ctc_loss(out, target, tuple(lengths), tuple(target_lens))
        loss.backward()
        for name, x in model.named_parameters(): #clamping gradients stabilizes CTC training
            if name not in ['fc1.weight', 'fc2.weight', 'fc1.bias', 'fc2.bias']:
                x.grad = x.grad.clamp(min = -1, max = 1)
        optimizer.step()
        end = time.time()
        #print("Network execution time: " + str(end-start))
        if batch_idx%100 == 0:
            print(loss.data)
        if batch_idx%200 == 0:
            print(labels[0])
            print(transcribe_output(out.cpu(), True))
        dataLoadStart = time.time()
    

In [17]:
def test(model): #unfinished
    model.eval()
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        formattedOut = torch.zeros([len(data), batch_size, len(ground_truth)])
        for i in range(0, len(data)):
            output = model(data[i])
            formattedOut[i] = output
        print(target[0])
        print(transcribe_output(formattedOut, True))
        
        

In [ ]:
for i in range(1,41):
    print("Beginning epoch " + str(i))
    scheduler.step()
    train(model, i)
    #test(model) 

Beginning epoch 1
tensor(32.7702, device='cuda:0')
MAD
(['l', 'x', 'l', 'w', 'w', 'k', 'l', 'x', 'k', 't', 'k', 'x', 'x', 'x', 'x', 'l', 'w', 'x', 'w', 'k', 'x', 'l', 'x', 'l', 't', 'x', 'l', 't', 't', 'w', 'x', 'k', 'x', 'k', 'w', 'x', 'k', 'x', 'k', 'x', 'x', 'k', 'l', 'x', 'k', 't', 'w', 'l', 'x', 'l', 'l', 'l', 'x', 'x', 'x', 't', 'x', 'x', 'x', 'x', 'w', 't', 'l', 'x', 'w', 'x', 'x', 'x', 'l', 'x', 'f', 'l', 'x', 'x', 'x', 'l', 'l', 'x', 't', 'l', 'w', 'x', 'x', 'x', 'x', 'l', 'x', 'x', 'x', 'x', 'k', 'l', 'w', 'x', 't', 'x', 'k', 'x', 'w', 't', 't', 'x', 'x', 'l', 'l', 'w', 'k', 'w', 'l', 'k', 'x', 'l', 'f', 'l', 'l', 'l', 'x', 'w', 'x', 't', 'w', 'k', 'k', 'x', 'l', 'l', 't', 'n', 'x', 't', 'x', 'x', 'l', 'l', 't', 'x', 'l', 't', 'l', 'x', 'k', 't', 'x', 'x', 'w', 'l', 'x', 'k', 't', 'l', 'k', 'x', 'x', 'k', 't', 'w', 'k', 't', 'x', 'l', 'w', 'k', 'k', 'w', 'l', 'x', 'k', 'l', 'k', 'k', 't', 'x', 'l', 'x', 'x', 't', 'l', 'x', 'c', 'k', 'x', 'x', 'w', 'x', 'w', 'x', 't', 'x', 'x'

tensor(5.4256, device='cuda:0')
tensor(4.7690, device='cuda:0')
IT HAS BEEN A PRETTY GAME OF LOVE FOR YOU AND THE EXCELLENT BALDOS
([], ['x', "''", 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'k', 'x', 'x', 'x', 'x', 't', 'x', 'x', 'k', 'i', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'q', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', "''", 'k', 't', 'x', 'x', 'i', 'x', "''", 'x', 'x', 'f', 'x', 't', 'x', 'i', 'x', 'k', 'x', 't', 'x', 't', 'i', 'k', 'x', 'l', 'x', 'x', 'k', 'l', 'i', 'x', 'x', 'x', 'x', 'i', 'x', 'x', 'x', 'x', 'i', 'x', 'x', 'x', 'x', 'x', 'i', 'x', 'x', 'x', 't', 'i', 'f', 'x', 'x', 'x', 't', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'f', 'x', 'x', 'x', 'i', 'i', 'x', 'i', 'k', 'x', 'x', 'x', 'k', 'x', 'x', 'x', 'l', 'l', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'l', 'x', 'x', 'l', 'x', 'x', 'w', 'x', 'f', 'x', 'l', 'x', 'x', 'x', 'x', 'x', 'k', 'i', 'x', 'x', 'x', 'x', 'x', 'x', 'i', 'x', 'w', 'f', 'x', 'i', 'x', "''", 'x', 'x', 'i', 'x', 'k', 'x', 'l', "''", 'x', 'x', 'x'

tensor(9.1853, device='cuda:0')
tensor(9.2819, device='cuda:0')
HAVE NO POWERS TO EXACT OBEDIENCE OR PUNISH DISOBEDIENCE TO THEIR RESOLUTIONS EITHER BY PECUNIARY MULCTS
([], ['f', 'i', 'x', 'i', 'a', 'i', 'x', 'i', 'x', 'x', 'i', 'i', 'i', "''", 'x', 't', 'i', 'i', 'i', 'i', 'i', 'q', 'i', 'i', 'x', 'h', 'i', 'x', 'y', 'w', 'i', 'q', 'x', 'i', 'i', 'i', 'j', 'i', 'i', 'x', 'i', 'i', 'x', 'x', 'i', 'i', 'i', 'i', 'f', "''", 'i', 'f', 'm', 'w', 'i', 'i', 'i', 'i', 'x', 'i', 'i', 'i', 'x', 'f', 'i', 'x', 'i', 'i', 'y', 'i', 'x', 'x', 'f', 'x', 'i', 'a', 'i', 'b', 'i', 'h', 'x', 'm', 'i', 'e', 'f', "''", 't', 'i', 'h', 'w', 'h', 'x', 'i', 'm', 'i', 'e', 'h', 'i', 'x', 'i', 'i', 'w', 'm', 'i', 'i', 'i', 'q', 'b', 'x', 'i', 'm', 'x', 'i', 'i', 'i', 't', 'i', 'i', 'i', 'i', 'i', 'x', 'i', 'i', 'i', 'x', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'q', 'i', 'i', 'x', 'i', 'y', 't', 'i', 'i', 'i', 'a', 'i', 'i', 'i', 'x', 'i', 'i', 'x', 'x', 'x', 'i', 'i', 'm', 'i', 'x', 'i', 'y', 'i', 'i', 'a', 'i', 'q

tensor(9.1066, device='cuda:0')
tensor(9.1781, device='cuda:0')
CANDIDE HAD NOT VIEWED HER AS YET WITH ATTENTION HIS THOUGHTS BEING ENTIRELY TAKEN UP WITH CUNEGONDE BUT RECOLLECTING HER AS SHE SPOKE
([], [' ', 'x', ' ', ' ', ' ', 'i', ' ', ' ', ' ', 'e', ' ', ' ', ' ', ' ', 'e', ' ', 'i', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'i', ' ', ' ', 'i', ' ', ' ', ' ', ' ', ' ', 't', ' ', ' ', ' ', 't', ' ', ' ', ' ', ' ', ' ', ' ', 'i', 'i', ' ', ' ', ' ', 'i', ' ', ' ', 'i', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'e', ' ', 'i', 'i', ' ', ' ', ' ', 'i', 'i', 'i', 'e', ' ', ' ', 'i', ' ', ' ', 'e', ' ', ' ', ' ', ' ', ' ', 'i', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'e', ' ', ' ', ' ', ' ', ' ', 'i', ' ', ' ', ' ', ' ', 'i', ' ', ' ', ' ', ' ', 'i', 'i', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'i', ' ', 'r', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'e', 'i', 'e', 'i', ' ', ' ', ' ', 'i', ' ', ' ', 'e', 't', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'i', 'e', ' ', ' ', ' ', ' ', ' ', 

tensor(8.9405, device='cuda:0')
tensor(8.8185, device='cuda:0')
SEASHELLS THAT ACTUALLY HOP AROUND TOP SHELL SNAILS RED HELMET SHELLS ANGEL WING CONCHS SEA HARES AND SO MANY OTHER EXHIBITS FROM THIS INEXHAUSTIBLE OCEAN
([], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 

...

tensor(8.7098, device='cuda:0')
tensor(8.7436, device='cuda:0')
RAN AS FAST AS THEY COULD DOWN TO THE SEASHORE WHILE BUMPO LEANED AGAINST THE WALL OF THE EMPTY DUNGEON SMILING AFTER THEM HAPPILY HIS BIG FACE SHINING LIKE POLISHED IVORY IN THE LIGHT OF THE MOON
([], [' ', ' ', ' ', ' ', ' ', ' ', ' ', 'e', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '

tensor(8.1833, device='cuda:0')
tensor(8.2036, device='cuda:0')
AND HOW EVEN IF YOU HAD THE TEMERITY
([], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'e', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'e', 'e', ' ', ' ', ' ', 'e', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'e', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'e', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'e', ' ', ' ', ' '

tensor(8.4900, device='cuda:0')
tensor(8.4570, device='cuda:0')
THEN KATHARINE GAVE IN ONCE AND FOR ALL WHAT YOU WILL HAVE IT NAMED SHE SAID
([], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'e', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '

tensor(8.8044, device='cuda:0')
tensor(8.6068, device='cuda:0')
IT WAS PLEASANT TO SEE HIS COMRADES GATHER ROUND HIM WITH SUCH HEARTY ADIEUS THAT HIS ONE HAND MUST HAVE TINGLED
([], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' 

tensor(8.8317, device='cuda:0')
tensor(8.8102, device='cuda:0')
LET US GIVE HIM ONLY HALF THE SUM PROMISED HIM WITH THE LETTER WITH AN AGREEMENT THAT HE SHALL HAVE THE OTHER HALF IN EXCHANGE FOR THE REPLY
([], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',

In [17]:
torch.save(model.state_dict(), './asr_long_test.t7')